<a href="https://colab.research.google.com/github/tranthuc2000/CS114.K21/blob/master/DectectSarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Người thực hiện:**
Trần Văn Thục,
Nguyễn Tri Đức Anh,
Trần Quốc Toàn

**Mô tả:**
Các bài toán Detect Sacasm trước đây hầu hết dùng sử dụng Tweeter để thu thập dữ liệu nhưng có nhiều thông tin gây nhiễu và khó phân biết nên đã có bộ dữ liệu này. Dữ liệu được lấy từ 2 website là TheOnion(tin châm biếm) và Huffpost( tin chính thống). Dataset có cải tiến là nó được viết bởi chuyên gia nên không có lỗi chính tả và formal hơn và nó ít nhiễu hơn do mỗi website chỉ chuyên về một mảng. Dataset gồm có link, headlines và một biến is_sarcastic=1 or =0.

**Import thư viện để liên kết với drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


mình đang có gì trong thư mục hiện tại

In [ ]:
ls

drive/        Sarcasm_Headlines_Dataset.json
sample_data/  Sarcasm_Headlines_Dataset_v2.json


In [ ]:
!unzip 'drive/My Drive/detectsarcasm/news-headlines-dataset-for-sarcasm-detection.zip'

Archive:  drive/My Drive/detectsarcasm/news-headlines-dataset-for-sarcasm-detection.zip
replace Sarcasm_Headlines_Dataset.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Sarcasm_Headlines_Dataset_v2.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


**Đọc file rồi gán vào biến dataframe**

In [ ]:
import pandas as pd
df_1 = pd.read_json("Sarcasm_Headlines_Dataset.json",lines=True)

In [ ]:
df_2 = pd.read_csv("drive/My Drive/detectsarcasm/datatest.csv",)
df_2

**xem thông tin của biến df_1**

In [ ]:
df_1.info()
print("                  ")
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26709 entries, 0 to 26708
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   article_link  26709 non-null  object
 1   headline      26709 non-null  object
 2   is_sarcastic  26709 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 626.1+ KB
                  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1937 entries, 0 to 1936
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Headline      1937 non-null   object
 1   Is_sarcastic  1937 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 30.4+ KB


**Xóa bỏ cột article link trong dataset**


In [ ]:
df_1=df_1.drop(labels="article_link",axis=1)

**tải về tệp tin giúp cho việc loại bỏ các thứ không quan trọng như dấu câu,...

Chạy vòng lập ép kiểu dữ liệu thành string để thao tác sau này tránh bị lỗi**

In [ ]:
import numpy as np
import string 
from string import digits
from string import punctuation
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
for i in df_1['headline']:
  i = str(i)

for a in df_2['Headline']:
  a = str(a)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Kiểm tra có gì trong df_1.Đã xóa link chưa

In [ ]:
df_1

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0
...,...,...
26704,american politics in moral free-fall,0
26705,america's best 20 hikes,0
26706,reparations and obama,0
26707,israeli ban targeting boycott supporters raise...,0


**Dọn dẹp các chữ số và chấm câu**

In [ ]:
hl_clean_1 = []
hl_clean_2 = []

for i in df_1['headline']:
    clean = i.translate(str.maketrans('', '', punctuation))
    #xóa kí hiệu đặc biệt như % & '
    clean = clean.translate(str.maketrans('', '', digits))
    #Xóa số
    hl_clean_1.append(clean)

for a in df_2['Headline']:
    clean = a.translate(str.maketrans('', '', punctuation))
    clean = clean.translate(str.maketrans('', '', digits))
    hl_clean_2.append(clean)


**Tách thành list**

In [ ]:
hl_tokens_1 = []
hl_tokens_2 = []
for i in hl_clean_1:
  i = i.split()
  #tách thành list
  hl_tokens_1.append(i)
for i in hl_clean_2:
  i = i.split()
  hl_tokens_2.append(i)  


**Tách từ trong tập train và tập test để những từ trùng nhau chuyển về số**


In [ ]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)
#tìm từ loại cho từng từ
# Init Lemmatizer 1
lemmatizer = WordNetLemmatizer()

hl_lemmatized_1 = []
for tokens in hl_tokens_1:
    lemm = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in tokens]
    hl_lemmatized_1.append(lemm)

 # Init Lemmatizer 2 
hl_lemmatized_2 = []
for tokens in hl_tokens_2:
    lemm = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in tokens]
    hl_lemmatized_2.append(lemm)

In [ ]:
hl_lemmatized_2

[['Judge',
  'Rejects',
  'White',
  'House',
  'Demand',
  'To',
  'Block',
  'John',
  'Boltons',
  'New',
  'Book'],
 ['Florida',
  'Governor',
  'Ripped',
  'For',
  'Trying',
  'To',
  'Pin',
  'COVID',
  'Spike',
  'On',
  'Hispanic',
  'Workers'],
 ['Congressional',
  'Races',
  'To',
  'Watch',
  'In',
  'New',
  'Yorks',
  'Democratic',
  'Primary'],
 ['The',
  'Coronavirus',
  'Pandemic',
  'Doesnt',
  'Care',
  'That',
  'Were',
  'All',
  'Bored',
  'With',
  'It'],
 ['Who',
  'Owns',
  'A',
  'River',
  'The',
  'Question',
  'Is',
  'Tearing',
  'This',
  'Community',
  'Apart'],
 ['Public',
  'Enemy',
  'Tells',
  'Dictator',
  'Trump',
  'To',
  '‘Shut',
  'The',
  'Fk',
  'Up',
  'In',
  'Explosive',
  'New',
  'Song'],
 ['If',
  'You',
  'Still',
  'Dont',
  'Get',
  'Why',
  'COVID',
  'Hit',
  'Black',
  'People',
  'Harder',
  'Read',
  'This'],
 ['Trump', 'Sets', 'The', 'Stage', 'For', 'Mayhem', 'At', 'Tulsa', 'Rally'],
 ['Melania',
  'Trump',
  'Posts',
  'Junete

**Vecto hóa 1 văn bản, giữ lại tối đa 2000 từ để dễ xử lí dữ liệu **

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
import numpy as np
max_features = 2000
max_token = len (hl_lemmatized_1)
tokenizer = Tokenizer (num_words= max_features)
tokenizer.fit_on_texts(hl_lemmatized_1)
sequences = tokenizer.texts_to_sequences(hl_lemmatized_1)
X_train = pad_sequences(sequences, maxlen = max_token)
# biến X_train thành dạng vector có dộ dài các vecto(mỗi vecto là 1 headline) có độ dài bằng nhau
#padding giúp các chuỗi trong X_train có độ dài bằng nhau

####

max_features = 2000
#để khớp 2 tập dữ liệu test
max_token = len (hl_lemmatized_1)
tokenizer = Tokenizer (num_words= max_features)
tokenizer.fit_on_texts(hl_lemmatized_2)
sequences = tokenizer.texts_to_sequences(hl_lemmatized_2)
X_test = pad_sequences(sequences, maxlen = max_token)
#padding giúp các chuỗi trong X_test có độ dài bằng nhau

Using TensorFlow backend.


In [ ]:
X_test

array([[   0,    0,    0, ...,  962,   11,  132],
       [   0,    0,    0, ...,  409,    7,  220],
       [   0,    0,    0, ...,   11,  147,  221],
       ...,
       [   0,    0,    0, ...,   79,   60,   38],
       [   0,    0,    0, ...,   29, 1441,    9],
       [   0,    0,    0, ...,    1,    5,    4]], dtype=int32)

In [ ]:
Y_train = df_1['is_sarcastic'].values
Y_train = np.vstack(Y_train)
# biến Y_train thành 1 ma trận
#X_trai,X_tes,Y_trai,Y_tes = train_test_split(X_train,Y_train,test_size=0.3, random_state = 42)
print (X_train)

#######
Y_test = df_2['Is_sarcastic'].values
Y_test = np.vstack(Y_test)
#biến Y_test thành 1 ma trận

[[   0    0    0 ... 1890    7 1985]
 [   0    0    0 ...  166    9  145]
 [   0    0    0 ...   41   14    1]
 ...
 [   0    0    0 ...    0    9   73]
 [   0    0    0 ...  686  715  425]
 [   0    0    0 ...  432    7    3]]


In [ ]:
len(X_test)

1937

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf  =  RandomForestClassifier(n_estimators=300)
#chọn model 
clf.fit(X_train, Y_train)
#train dữ liệu, đưavào model để lựa chọn ra công thức phù hợp

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
from sklearn.preprocessing import LabelEncoder,MinMaxScaler

In [ ]:
le = LabelEncoder()
y_result=le.fit_transform(Y_test)
y_pred=clf.predict(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


acuracy của dữ liệu khi chưa xử lý xong. Chỉ số tạm chấp nhận được

In [ ]:
print("Result: ", (y_pred == y_result).tolist().count(True)/len(y_result))

Result:  0.610738255033557


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_result,y_pred))

              precision    recall  f1-score   support

           0       0.58      0.75      0.65       938
           1       0.67      0.48      0.56       999

    accuracy                           0.61      1937
   macro avg       0.62      0.62      0.61      1937
weighted avg       0.63      0.61      0.60      1937



Tiếp tục xử lý

In [ ]:
from sklearn.model_selection import train_test_split
Y_train = df_1['is_sarcastic'].values
Y_train = np.vstack(Y_train)
# biến Y_train thành 1 ma trận
X_train,X_test,Y_train,Y_test = train_test_split(X_train,Y_train,test_size=0.3, random_state = 42)
print (X_train)

[[   0    0    0 ...  647    2   22]
 [   0    0    0 ...  309   75  571]
 [   0    0    0 ...  218    7  824]
 ...
 [   0    0    0 ...  105    9  363]
 [   0    0    0 ...    5    2 1127]
 [   0    0    0 ... 1663   22    3]]


In [ ]:
len(X_test)

8013

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf  =  RandomForestClassifier(n_estimators=300)
#chọn model 
clf.fit(X_train, Y_train)
#train dữ liệu



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
from sklearn.preprocessing import LabelEncoder,MinMaxScaler

In [ ]:
le = LabelEncoder()
y_result=le.fit_transform(Y_test)
y_pred=clf.predict(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
print("Result: ", (y_pred == y_result).tolist().count(True)/len(y_result))

Result:  0.6558093098714589


Đối chiếu performance của model trên dataset đã cho và trên 2000 headine mới ta thấy có sự chênh lệch, 0.61 và 0.66. Performance khi chạy trên dataset cao hơn(0.66), còn khi chạy trên 2000 headline mới thì thấp hơn do phải lược bỏ nhiều từ

In [ ]:
#sacarsm detection cho một headline bất kỳ được nhập vào
str1 = str(input("Nhap chuoi: "))

Nhap chuoi: i am very happy hihi


In [ ]:
test =[]
clean = str1.translate(str.maketrans('', '', punctuation))
clean = clean.translate(str.maketrans('', '', digits))
test.append(clean)

testn=[]
for i in test:
    i = i.split()
    testn.append(i)

testn_1 = []
for i in testn:
    lemm = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in i]
    testn_1.append(lemm)

max_features = 2000
max_token = len(hl_lemmatized_1)
tokenizer = Tokenizer (num_words= max_features)
tokenizer.fit_on_texts(testn_1)
sequences = tokenizer.texts_to_sequences(testn_1)
X_test_1 = pad_sequences(sequences, maxlen = max_token)

In [ ]:
y_pred=clf.predict(X_test_1)
kq = y_pred[0]
if (kq != 0):
  print("Not sarcasm")
else:
  print("Sarcasm")

Sarcasm


Đã cho ra kết quả khá khả quan

In [ ]:
f1 = read;
f1 = f1[:1000]

**save model với pickle**


In [ ]:
import pickle 
pickle.dump(clf,open('/content/drive/My Drive/Colab Notebooks/pickle/pickle.docx', 'wb'))
